In [36]:
import glob
import os
txt_file_path = './aiffel/lyricist/archive (1)/*' # 파일 path
txt_list = glob.glob(txt_file_path) # 읽어올 파일들의 리스트. 여러개의 파일을 다룰 때 사용
raw_corpus = [] # 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list: # 파일 리스트 중에서 파일 하나를 선택
    with open(txt_file, "r", encoding='UTF-8') as f: #파일 open
        raw = f.read().splitlines() #한 줄씩 읽기, \n 제거
        raw_corpus.extend(raw) #해당 내용을 raw_corpus에 추가
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


In [28]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \\1 ", sentence)  # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)   # 공백이 여러개여도 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'      # 문장 앞뒤로 <start>와 <end>를 추가
    return sentence
print(preprocess_sentence("This @_is ;;;sample        sentence."))  

<start> this is sample sentence <end>


In [29]:
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue # 문장 길이가 0이면 패스
    if sentence[-1] == ":": continue #  문장 마지막이 : 여도 패스
    corpus.append(preprocess_sentence(sentence))
corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby can t you read the signs i won t bore you with the details baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

In [30]:
import tensorflow as tf

def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축
    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축
    tensor = tokenizer.texts_to_sequences(corpus)   
    # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰짐
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=15, padding='post')  
    print(tensor,tokenizer)
    return tensor, tokenizer
tensor, tokenizer = tokenize(corpus)

[[  2 300  26 ...   0   0   0]
 [  2 217  12 ...   0   0   0]
 [  2  22  16 ...   0   0   0]
 ...
 [  2  21  73 ...   0   0   0]
 [  2  40  24 ...   0   0   0]
 [  2  21  73 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x0000021A1AD07430>


In [41]:
import numpy as np
# 추가 처리
tensor_cor = []
for sentence in tensor:
    if sentence[0] != 2: continue # 문장 제일 앞 부분이 <start>가 아닐 때 패스
    if sentence[-1] != 0: continue # 문장 제일 끝 부분이 패딩된 값이 아닐 때 패스
    tensor_cor.append(sentence)
tensor = np.array(tensor_cor)

In [42]:
tensor.shape

(155339, 15)

In [43]:
src_input = tensor[:, :-1]  
# tensor에서 마지막 토큰(여기선 0)을 잘라내서 소스 문장을 생성합니다. 

tgt_input = tensor[:, 1:]    
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
print(src_input[0])
print(tgt_input[0])

[   2  300   26   95 4806    3    0    0    0    0    0    0    0    0]
[ 300   26   95 4806    3    0    0    0    0    0    0    0    0    0]


In [44]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1    # 0:<pad>를 포함하여 7001개
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [45]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

In [46]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124271, 14)
Target Train: (124271, 14)


In [47]:
class TextGenerator(tf.keras.Model): # 텍스트 생성기
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        return out
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [48]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-4.19513890e-06, -1.09222921e-04, -4.35716356e-05, ...,
          2.05335600e-04, -1.55378966e-05, -3.50236456e-04],
        [ 2.34603736e-04,  2.80564564e-04,  9.44306084e-05, ...,
          1.60920157e-04, -9.56527765e-06, -2.77408137e-04],
        [ 4.57151793e-04,  5.22724353e-04,  3.33764940e-04, ...,
          1.98872629e-04, -9.38648300e-05, -3.52247094e-04],
        ...,
        [-1.02158356e-03,  8.07036922e-05,  3.46783287e-04, ...,
         -1.53525942e-03, -9.50414047e-04,  1.65737118e-04],
        [-1.10295205e-03,  3.95190960e-04,  4.74115484e-04, ...,
         -1.65818527e-03, -1.06344977e-03,  1.28114712e-04],
        [-1.13764009e-03,  6.64822175e-04,  5.81965782e-04, ...,
         -1.73982256e-03, -1.15699472e-03,  6.22093794e-05]],

       [[-4.19513890e-06, -1.09222921e-04, -4.35716356e-05, ...,
          2.05335600e-04, -1.55378966e-05, -3.50236456e-04],
        [ 7.88407415e-05, -1.91896004e-04, -7

In [49]:
model.summary()

Model: "text_generator_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_6 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_7 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_3 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [51]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)
model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
606/606 [==============================] - 1783s 3s/step - loss: 3.3378
Epoch 2/10
606/606 [==============================] - 1784s 3s/step - loss: 2.6966
Epoch 3/10
606/606 [==============================] - 1833s 3s/step - loss: 2.4185
Epoch 4/10
606/606 [==============================] - 1790s 3s/step - loss: 2.1921
Epoch 5/10
606/606 [==============================] - 1821s 3s/step - loss: 1.9884
Epoch 6/10
606/606 [==============================] - 1804s 3s/step - loss: 1.8139
Epoch 7/10
606/606 [==============================] - 1776s 3s/step - loss: 1.6545
Epoch 8/10
606/606 [==============================] - 1771s 3s/step - loss: 1.5107
Epoch 9/10
606/606 [==============================] - 1774s 3s/step - loss: 1.3863
Epoch 10/10
606/606 [==============================] - 1773s 3s/step - loss: 1.2829


In [52]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됨 
        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙임
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장

In [53]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he s a highway chile <end> '

In [54]:
generate_text(model, tokenizer, init_sentence="<start> love is")

'<start> love is a beautiful thing <end> '

In [55]:
generate_text(model, tokenizer, init_sentence="<start> your")

'<start> your smile fades in the summer <end> '

In [57]:
generate_text(model, tokenizer, init_sentence="<start> my")

'<start> my girl <end> '

In [59]:
generate_text(model, tokenizer, init_sentence="<start> never")

'<start> never thought i d see your face again <end> '

In [60]:
generate_text(model, tokenizer, init_sentence="<start> baby")

'<start> baby i m from new york <end> '

In [67]:
generate_text(model, tokenizer, init_sentence="<start> rainy")

'<start> rainy night i ll be waiting <end> '

In [76]:
generate_text(model, tokenizer, init_sentence="<start> wanna")

'<start> wanna argue all day hangin out all night <end> '